In [ ]:
import json
import matplotlib.pyplot as plt

last_req = {}
all_req = {}

with open('./data/data_training/dataSet_3/requests', 'r') as file:
    for line in file.readlines():
        data = json.loads(line)
        key = data['metaKey']
        # first key
        if key not in last_req.keys():
            last_req[key] = data['startTime'] + data['durationsInMs']
            continue

        l_req = last_req[key]
        last_req[key] = max(l_req, data['startTime'] + data['durationsInMs'])
        if l_req > data['startTime']:
            continue

        if key not in all_req.keys():
            all_req[key] = []
        all_req[key].append(data['startTime'] - l_req)

fig_num = len(all_req.keys())
cur = 1
plt.figure(figsize=(6, 400))
for key, value in all_req.items():
    plt.subplot(fig_num, 1, cur)
    plt.plot(value)
    plt.title(key)
    cur += 1
plt.subplots_adjust(hspace=0.4)
plt.show()

In [ ]:
#每个时间来的请求
import json
import matplotlib.pyplot as plt

timeline = {}

base_start_time = -1
with open('./data/data_training/function_smoke_test/requests', 'r') as file:
    for line in file.readlines():
        data = json.loads(line)
        key = data['metaKey']
        start_time = data['startTime']
        if base_start_time == -1:
            base_start_time = start_time
        if key not in timeline.keys():
            timeline[key] = {}
        if (start_time - base_start_time) not in timeline[key].keys():
            timeline[key][start_time - base_start_time] = 1
        else:
            timeline[key][start_time - base_start_time] += 1

fig_num = len(timeline.keys())
cur = 1
plt.figure(figsize=(6, 500))
for key, value in timeline.items():
    x = list(value.keys())
    y = list(value.values())
    plt.subplot(fig_num, 1, cur)
    plt.plot(x, y)
    plt.title(key)
    cur += 1
plt.subplots_adjust(hspace=0.4)
plt.show()

In [6]:
#并发度
import json
import heapq
import matplotlib.pyplot as plt
from tqdm import tqdm

cur_req = {}
last_req = {}
cur_time = 1685980824304

with open('./data/data_training/function_smoke_test/metas', 'r') as file:
    for line in file.readlines():
        data = json.loads(line)
        key = data['key']
        cur_req[key] = []
        last_req[key] = []

with open('./data/data_training/function_smoke_test/requests', 'r') as file:
    for line in tqdm(file.readlines()):
        data = json.loads(line)
        key = data['metaKey']
        while cur_time + 1000 < data['startTime']:
            for key, value in last_req.items():
                cur_req[key].append(len(last_req[key]))
                while len(last_req[key]) > 0:
                    latest_change_time = heapq.heappop(last_req[key])
                    if len(last_req[key]) > 0 and latest_change_time <= cur_time:
                        latest_change_time = heapq.heappop(last_req[key])
                    else:
                        heapq.heappush(last_req[key], latest_change_time)
                        break
            cur_time += 1000
        heapq.heappush(last_req[key], data['startTime'] + data['durationsInMs'])

fig_num = len(cur_req.keys())
cur = 1
plt.figure(figsize=(100, 200))
for key, value in cur_req.items():
    plt.subplot(fig_num, 1, cur)
    plt.plot(value)
    plt.title(key)
    cur += 1
plt.subplots_adjust(hspace=0.5)
plt.show()

100%|██████████| 980/980 [00:00<00:00, 299069.99it/s]


: 